In [30]:
import re
import os.path
from markdown import markdown
from os import listdir
from os.path import isfile, join


class Renderer:
    def __init__(self, template_folder, template_path, content_folder, content_path, output_path):
        self.template_path = template_path
        self.content_path = content_path
        self.template_folder = template_folder
        self.output_path = output_path
        self.content_folder = content_folder
    
    
    def get_file_content(self, path):
        file = open(path, 'r') 
        content = file.read()
        file.close()
        return content
    
    
    def post_links(self):
        # get posts directory
        files = [f for f in listdir(self.content_folder) if isfile(join(self.content_folder, f))]
    
        # read all files, get titles and dates
        ret = []
        for file in files:
            file_context = self.read_content(join(self.content_folder, file))

            # if there is no title in the context, this is not a post, skip it
            if 'title' not in file_context:
                continue
            
            ret.append((file, file_context['title'], file_context['date']))
        
        return self.render_post_links(ret)
       
        
    def render_post_links(self, posts):        
        ret = []
        for p in posts:
            ret.append(f'<a href="{p[0].replace(".md", ".html")}">{p[1]}</a>')
        return ''.join(ret)
    
        
    def read_content(self, path):
        # collect variables now
        file = open(path, 'r')
        lines = file.readlines()
        file.close()
        
        # read lines, get the ones starting with
        context = {}
        content = []
        
        for line in lines:
            if line[0] == '@':
                name, val = line[1:].split(':')
                context[name] = val.strip()
            else:
                content.append(line)
                
        context['content'] = markdown(''.join(content), extensions=['codehilite'])
        
        context['post_links'] = self.post_links
        
        return context
    
    
    def render_template(self):
        context = self.read_content(self.content_path)
        template_content = self.get_file_content(self.template_path)
        
        # find <%SYMBOL%>
        symbols = re.findall(r"<\%(.*?)\%>", template_content)
        symbols = [k.replace('<%', '').replace('%>', '') for k in symbols]
        
        # resolve symbols with variables
        for symbol in symbols:
            # if symbol is file
            file_path = f'{self.template_folder}\\{symbol}.html'
            if os.path.isfile(file_path):
                # read file content and replace
                template_content = template_content.replace(f'<%{symbol}%>', self.get_file_content(file_path))
            elif symbol in context:
                # get val from context
                value = None
                
                if callable(context[symbol]):
                    value = context[symbol]()
                else:
                    value = context[symbol]
    
                template_content = template_content.replace(f'<%{symbol}%>', value)
            else:
                # throw error
                raise Exception(f'Symbol cannot be resolved. No file "{file_path}", No variable in content as "{symbol}"')
        
        return template_content
    
    
    def save_html(self):
        file = open(self.output_path, 'w')
        file.write(self.render_template())
        file.close()

In [31]:
# render one post
renderer = Renderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\post.html',
    content_folder  = '.\\posts',
    content_path    = '.\\posts\\datalake-paths.md',
    output_path     = '.\\output\datalake-paths.html'
)

In [32]:
renderer.save_html()

In [33]:
# render index
#   read context from a template fu
#
#

renderer = Renderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\index.html',
    content_folder  = '.\\posts',
    content_path    = '.\\posts\\index.md',
    output_path     = '.\\output\index.html'
)

In [34]:
renderer.save_html()

UnboundLocalError: local variable 'file_context' referenced before assignment

In [21]:
# render index
#   read context from a template fu
#
#

renderer = Renderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\post.html',
    content_folder  = '.\\posts',
    content_path    = '.\\posts\\about.md',
    output_path     = '.\\output\\about.html'
)

In [22]:
renderer.save_html()